# Importing necessary libraries

 This Python code snippet sets the OpenAI API key, which is required for making requests to the OpenAI API. It assigns the API key to the environment variable ‘OPENAI_API_KEY’. The ‘openai.api_key’ is then set to the value of the environment variable. This allows the code to authenticate and interact with the OpenAI services.

In [15]:
import os
import openai
import json
os.environ['OPENAI_API_KEY']=''
openai.api_key=os.environ['OPENAI_API_KEY']

In [16]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

# OCR 

OCR (Optical Character Recognition) Data Extraction is a technology that enables machines to extract text from scanned documents, images, or other sources. It allows us to convert this extracted text into digital data, making it accessible and editable. OCR software identifies letters in an image, constructs words, and then assembles sentences, eliminating the need for manual data entry and improving efficiency

# Working of the Code 

Over here, we define a OCR function, where we take the image as the input. There exists a subscription_key which we get from Microsoft Azure Services. The image can be of any extension, but it can be passed in the form of either bytes directly or in the form of a string. The open(image,'rb') function decodes the file path into one having the bytes format. We get the status of the operation in the vairable status_code and the results of the operation in ocr_out. 

The provided code extracts a URL from the response headers, sends a GET request to that URL to monitor the status of an OCR (Optical Character Recognition) process. It repeatedly checks the status until it changes from "running," pausing for 1 second between checks. Once the OCR is complete, it retrieves the final output in JSON format.

In [17]:
import requests
subscription_key = "d85c088ad95a42ff8641d0440486311f"
def ocr(img,mime_type='image/jpeg'):

    """

    Ocr extraction from Image or Image at local path

    Input: Image or Image local path

    Output: Coginitive Services OCR output response json

    """

    #url = "https://centralindia.api.cognitive.microsoft.com/vision/v3.2/read/analyze"

    url = "https://centralindia.api.cognitive.microsoft.com/vision/v3.2/read/analyze?language=en"

    payload={}

    img_bytes=None

    if type(img)==str:

        files=[('image',('tmp.jpg',open(img,"rb"),mime_type))]

    else:

        img_bytes=img

        files=[('image',('tmp.jpg',img_bytes,mime_type))]

    headers = {'Ocp-Apim-Subscription-Key': subscription_key}

    response = requests.request("POST", url, headers=headers, data=payload, files=files, verify=False)

    status_code = response.status_code

    ocr_out=response.headers

    if status_code== 202:

        url=ocr_out['Operation-Location']

        

        response = requests.request("GET", url, headers=headers, data=payload, verify=False)

        ocr_out=response.json()

        while ocr_out["status"]=="running":

            time.sleep(1)

            response = requests.request("GET", url, headers=headers, data=payload, verify=False)

            ocr_out=response.json()


    return dict(ocr_out),img_bytes

# IMAGE TO JSON AND USING OPENAI

We now define the Image_to_JSON function, which calls the ocr(image_path) function inside it. It returns 2 values namely ocr_out, which contains the details of the model and the image in the bytes format. these 2 values are stored in the 2 variables namely results, img_bytes. we create a new datafram called data from result which has 2 parts called 'analyzeResult' and 'readResults'.

1. The code iterates through each item in the data (which represents OCR results).
2. For each item, it further iterates through each line within that item.
3. It extracts the 'text' value from each line and adds it to the `texts` list.
4. In simple terms, it collects all the recognized text lines from the image and stores them in the `texts` list.


In [18]:
def Image_to_JSON(image_path):
    
    result,img_bytes = ocr(image_path)
    data=result['analyzeResult']['readResults']
    texts=[line['text'] for item in data for line in item['lines']]

    ocr_string = " ".join(texts)  # Stores the OCR content extracted from the image in a string which can be fed into ChatGPT

    # Extract the text from the OCR result and concatenate it to ocr_string
    #for i in range(len(result[0])):
        #ocr_string = ocr_string + result[0][i][1][0] + " "
    # Define the string that specifies the required information
    Information_String = "Name of the Airline as AIRLINE_NAME, Name of the Passenger as NAME, Flight Number as FLIGHT_NUM, Departure City as DEP_CITY, Destination City as DES_CITY, Date of Departure as DOD"

    # Create a query for ChatGPT, including the OCR content and the required information
    # ChatGPT_Query = f"Understand this data, it's from the boarding pass of a flight ticket, so that should be the context necessary to understand it: {ocr_string} \nFrom the information provided above, answer the following in a JSON format with no other explanation, just the JSON code: {Information_String}. Expand any city acronyms you encounter with data provided above. Format the Departure Date in YYYY-MM-DD. The Departure City and Destination City should be the full names of the cities."
    ChatGPT_Query=f"""The following is a raw dump of an OCR output from a boarding pass of a flight. Extract the airline name,name of the passenger,flight number,departure city,destination city and date of departure from it. Format the Departure Date in YYYY-MM-DD. The Departure City and Destination City should be in their airport codes. The raw OCR data of boarding pass is {ocr_string}.
    Output Format:
    AIRLINE_NAME:<insert airline name>
    PASSENGER_NAME:<insert passenger name>
    FLIGHT_NUM:<insert flight number>
    DEP_CITY:<insert departure airport code>
    DES_CITY:<insert destination airport code>
    DOD:<insert date of departure>
    """
    def get_completion(prompt, model="gpt-3.5-turbo-instruct"):
        # Prepare the prompt in the required format for ChatGPT

        # Get the completion from ChatGPT
        response = openai.Completion.create(
            model=model,
            prompt=prompt,
            max_tokens=3500,
            temperature=0
        )

        # Extract the content of the response
        return response

    # Get the completion from ChatGPT based on the query
    Output = get_completion(ChatGPT_Query)
    # Convert the JSON string to a Python object
    data = Output["choices"][0]["text"]

    # Write the JSON object to a file with indentation
    return data,Output["usage"]["prompt_tokens"],Output["usage"]["completion_tokens"]

In [19]:
image_path = r"C:\\Users\\pinak.mahapatra\\Downloads\\boarding_pass.jpg"
b, c, d = Image_to_JSON(image_path)
print(b)

APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [10]:
data = dict(line.split(': ') for line in b.strip().split('\n'))
json_data = json.dumps(data, indent=4)

print(json_data)

{
    "AIRLINE_NAME": "SINGAPORE AIRLINES",
    "PASSENGER_NAME": "HENG KOK HONG MR",
    "FLIGHT_NUM": "SQ 231",
    "DEP_CITY": "SIN",
    "DES_CITY": "SYD",
    "DOD": "2018-01-19"
}


In [ ]:
print(c,d)
print(c*0.0015/1000 + d*0.0020/1000)

In [ ]:
# C:\suresh\Boarding pass images\singapore.jpg